In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import  train_test_split
from keras.utils.np_utils import to_categorical
from keras import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras import regularizers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
jantung = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Capstone Project Bangkit/heart.csv")
jantung.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
cols_to_normalized = ['age', 'trestbps','chol', 'thalach', 'thal']
jantung_normalized = jantung[cols_to_normalized].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

jantung_normalized['sex'] = jantung['sex']
jantung_normalized['cp'] = jantung['cp']
jantung_normalized['fbs'] = jantung['fbs']
jantung_normalized['restecg'] = jantung['restecg']
jantung_normalized['exang'] = jantung['exang']
jantung_normalized['oldpeak'] = jantung['oldpeak']
jantung_normalized['slope'] = jantung['slope']
jantung_normalized['ca'] = jantung['ca']
jantung_normalized['target'] = jantung['target']
jantung_normalized

,age,trestbps,chol,thalach,thal,sex,cp,fbs,restecg,exang,oldpeak,slope,ca,target
0,0.708333,0.481132,0.244292,0.603053,0.333333,1,3,1,0,0,2.3,0,0,1
1,0.166667,0.339623,0.283105,0.885496,0.666667,1,2,0,1,0,3.5,0,0,1
2,0.250000,0.339623,0.178082,0.770992,0.666667,0,1,0,0,0,1.4,2,0,1
3,0.562500,0.245283,0.251142,0.816794,0.666667,1,1,0,1,0,0.8,2,0,1
4,0.583333,0.245283,0.520548,0.702290,0.666667,0,0,0,1,1,0.6,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.583333,0.433962,0.262557,0.396947,1.000000,0,0,0,1,1,0.2,1,0,0
299,0.333333,0.150943,0.315068,0.465649,1.000000,1,3,0,1,0,1.2,1,0,0
300,0.812500,0.471698,0.152968,0.534351,1.000000,1,0,1,1,0,3.4,1,2,0
301,0.583333,0.339623,0.011416,0.335878,1.000000,1,0,0,1,1,1.2,1,1,0


In [ ]:
jantung_normalized.isnull().any()

age         False
trestbps    False
chol        False
thalach     False
thal        False
sex         False
cp          False
fbs         False
restecg     False
exang       False
oldpeak     False
slope       False
ca          False
target      False
dtype: bool

In [ ]:
#Splitting data into feature and label
X = jantung_normalized.drop("target", 1)
y = jantung_normalized["target"]

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [ ]:
Y_train = to_categorical(y_train, num_classes=None)
Y_test = to_categorical(y_test, num_classes=None)

In [ ]:
Y_train_binary = y_train.copy()
Y_test_binary = y_test.copy()

Y_train_binary[Y_train_binary > 0] = 1
Y_test_binary[Y_test_binary > 0] = 1

In [ ]:
# Construct model
model = Sequential()
model.add(Dense(2048,activation = 'relu',input_shape=(13,))) 
model.add(Dropout(0.71))
model.add(Flatten())
model.add(Dense(1024,activation = 'relu'))
model.add(Dropout(0.44))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.71)) 
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.44)) 
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.71)) 
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.44)) 
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.71)) 
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [ ]:
# Display model architecture summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2048)              28672     
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.92):
      print("\nAkurasi telah mencapai >92%!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
history=model.fit(X_train, Y_train_binary, validation_data=(X_test, Y_test_binary), epochs=100, batch_size=10, verbose=True)

Epoch 1/100
25/25 [==============================] - 2s 52ms/step - loss: 0.8258 - accuracy: 0.4541 - val_loss: 0.6940 - val_accuracy: 0.5246
Epoch 2/100
25/25 [==============================] - 1s 30ms/step - loss: 0.8201 - accuracy: 0.4794 - val_loss: 0.6915 - val_accuracy: 0.5246
Epoch 3/100
25/25 [==============================] - 1s 30ms/step - loss: 0.8422 - accuracy: 0.4243 - val_loss: 0.6918 - val_accuracy: 0.5246
Epoch 4/100
25/25 [==============================] - 1s 30ms/step - loss: 0.7650 - accuracy: 0.4756 - val_loss: 0.6922 - val_accuracy: 0.5246
Epoch 5/100
25/25 [==============================] - 1s 28ms/step - loss: 0.6967 - accuracy: 0.5565 - val_loss: 0.6884 - val_accuracy: 0.5246
Epoch 6/100
25/25 [==============================] - 1s 29ms/step - loss: 0.7694 - accuracy: 0.5100 - val_loss: 0.6907 - val_accuracy: 0.5246
Epoch 7/100
25/25 [==============================] - 1s 31ms/step - loss: 0.7048 - accuracy: 0.5446 - val_loss: 0.6883 - val_accuracy: 0.5246
Epoch 